# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/bank-additional-full.csv', sep = ';')

In [3]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



In [4]:
from pandas.api.types import is_numeric_dtype

def list_uniques(df):
    for column in df.columns:
        if not is_numeric_dtype(df[column]):
            print(df[column].value_counts())
            print('\n---\n')

In [5]:
list_uniques(df)

job
admin.           10422
blue-collar       9254
technician        6743
services          3969
management        2924
retired           1720
entrepreneur      1456
self-employed     1421
housemaid         1060
unemployed        1014
student            875
unknown            330
Name: count, dtype: int64

---

marital
married     24928
single      11568
divorced     4612
unknown        80
Name: count, dtype: int64

---

education
university.degree      12168
high.school             9515
basic.9y                6045
professional.course     5243
basic.4y                4176
basic.6y                2292
unknown                 1731
illiterate                18
Name: count, dtype: int64

---

default
no         32588
unknown     8597
yes            3
Name: count, dtype: int64

---

housing
yes        21576
no         18622
unknown      990
Name: count, dtype: int64

---

loan
no         33950
yes         6248
unknown      990
Name: count, dtype: int64

---

contact
cellular     26144
telep

### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

The business objective of our task is to aid this banking institution in targeting customers who are likely to subscribe to this bank's term deposit account. As shown below, marketing campaigns of this type do not convert a large percentage of users; only about 11% of customers in this campaign went through with a subscription. We have lots of information about the demographics and financial information of targeted users, as well as information about when the potential customer was contacted. Our goal is to uncover patterns in the data, allowing the bank to more effectively choose who to contact, which hopefully in the future will lead to a higher percentage of customers successfully converted (in future campaigns).

### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features, prepare the features and target column for modeling with appropriate encoding and transformations.

Before dealing with the categorical features, I want to scale the numerical features using StandardScaler:

In [7]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

In [8]:
numerical_columns = df.select_dtypes(include='number')
categorical_columns = df.select_dtypes(exclude='number')

In [9]:
scaler.fit(numerical_columns)
scaled_num_columns = scaler.transform(numerical_columns)
scaled_numerical_columns = pd.DataFrame(scaled_num_columns, columns=scaler.get_feature_names_out())

In [10]:
categorical_columns

,job,marital,education,default,housing,loan,contact,month,day_of_week,poutcome,y
0,housemaid,married,basic.4y,no,no,no,telephone,may,mon,nonexistent,no
1,services,married,high.school,unknown,no,no,telephone,may,mon,nonexistent,no
2,services,married,high.school,no,yes,no,telephone,may,mon,nonexistent,no
3,admin.,married,basic.6y,no,no,no,telephone,may,mon,nonexistent,no
4,services,married,high.school,no,no,yes,telephone,may,mon,nonexistent,no
...,...,...,...,...,...,...,...,...,...,...,...
41183,retired,married,professional.course,no,yes,no,cellular,nov,fri,nonexistent,yes
41184,blue-collar,married,professional.course,no,no,no,cellular,nov,fri,nonexistent,no
41185,retired,married,university.degree,no,yes,no,cellular,nov,fri,nonexistent,no
41186,technician,married,professional.course,no,no,no,cellular,nov,fri,nonexistent,yes


In [11]:
scaled_numerical_columns

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
0,1.533034,0.010471,-0.565922,0.195414,-0.349494,0.648092,0.722722,0.886447,0.712460,0.331680
1,1.628993,-0.421501,-0.565922,0.195414,-0.349494,0.648092,0.722722,0.886447,0.712460,0.331680
2,-0.290186,-0.124520,-0.565922,0.195414,-0.349494,0.648092,0.722722,0.886447,0.712460,0.331680
3,-0.002309,-0.413787,-0.565922,0.195414,-0.349494,0.648092,0.722722,0.886447,0.712460,0.331680
4,1.533034,0.187888,-0.565922,0.195414,-0.349494,0.648092,0.722722,0.886447,0.712460,0.331680
...,...,...,...,...,...,...,...,...,...,...
41183,3.164336,0.292025,-0.565922,0.195414,-0.349494,-0.752343,2.058168,-2.224953,-1.495186,-2.815697
41184,0.573445,0.481012,-0.565922,0.195414,-0.349494,-0.752343,2.058168,-2.224953,-1.495186,-2.815697
41185,1.533034,-0.267225,-0.204909,0.195414,-0.349494,-0.752343,2.058168,-2.224953,-1.495186,-2.815697
41186,0.381527,0.708569,-0.565922,0.195414,-0.349494,-0.752343,2.058168,-2.224953,-1.495186,-2.815697


In [12]:
scaled_df = pd.concat([scaled_numerical_columns, categorical_columns], axis=1)

In [13]:
scaled_df

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,...,marital,education,default,housing,loan,contact,month,day_of_week,poutcome,y
0,1.533034,0.010471,-0.565922,0.195414,-0.349494,0.648092,0.722722,0.886447,0.712460,0.331680,...,married,basic.4y,no,no,no,telephone,may,mon,nonexistent,no
1,1.628993,-0.421501,-0.565922,0.195414,-0.349494,0.648092,0.722722,0.886447,0.712460,0.331680,...,married,high.school,unknown,no,no,telephone,may,mon,nonexistent,no
2,-0.290186,-0.124520,-0.565922,0.195414,-0.349494,0.648092,0.722722,0.886447,0.712460,0.331680,...,married,high.school,no,yes,no,telephone,may,mon,nonexistent,no
3,-0.002309,-0.413787,-0.565922,0.195414,-0.349494,0.648092,0.722722,0.886447,0.712460,0.331680,...,married,basic.6y,no,no,no,telephone,may,mon,nonexistent,no
4,1.533034,0.187888,-0.565922,0.195414,-0.349494,0.648092,0.722722,0.886447,0.712460,0.331680,...,married,high.school,no,no,yes,telephone,may,mon,nonexistent,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,3.164336,0.292025,-0.565922,0.195414,-0.349494,-0.752343,2.058168,-2.224953,-1.495186,-2.815697,...,married,professional.course,no,yes,no,cellular,nov,fri,nonexistent,yes
41184,0.573445,0.481012,-0.565922,0.195414,-0.349494,-0.752343,2.058168,-2.224953,-1.495186,-2.815697,...,married,professional.course,no,no,no,cellular,nov,fri,nonexistent,no
41185,1.533034,-0.267225,-0.204909,0.195414,-0.349494,-0.752343,2.058168,-2.224953,-1.495186,-2.815697,...,married,university.degree,no,yes,no,cellular,nov,fri,nonexistent,no
41186,0.381527,0.708569,-0.565922,0.195414,-0.349494,-0.752343,2.058168,-2.224953,-1.495186,-2.815697,...,married,professional.course,no,no,no,cellular,nov,fri,nonexistent,yes


In [14]:
scaled_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  float64
 1   duration        41188 non-null  float64
 2   campaign        41188 non-null  float64
 3   pdays           41188 non-null  float64
 4   previous        41188 non-null  float64
 5   emp.var.rate    41188 non-null  float64
 6   cons.price.idx  41188 non-null  float64
 7   cons.conf.idx   41188 non-null  float64
 8   euribor3m       41188 non-null  float64
 9   nr.employed     41188 non-null  float64
 10  job             41188 non-null  object 
 11  marital         41188 non-null  object 
 12  education       41188 non-null  object 
 13  default         41188 non-null  object 
 14  housing         41188 non-null  object 
 15  loan            41188 non-null  object 
 16  contact         41188 non-null  object 
 17  month           41188 non-null 

The data seems pretty well put together; there are no missing values, and looking through the categorical columns above, there are no extraneous categories. This dataset is far easier to work with than the used cars dataset in the previous practical application assignment, which had a lot of junk values that were a pain to work through.

Simple one-hot encoding should work to prepare the features.

In [15]:
X = pd.get_dummies(scaled_df.drop(columns='y'))

In [16]:
y = scaled_df['y']

### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

I am a little confused by this; doesn't using k-fold cross-validation make splitting the data into a training and testing set manually unnecessary?

I will do it anyway, as requested:

In [17]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

A large majority of the target values are 'No'; therefore, a dummy classifier that simply predicts 'No' in all cases receives a high baseline accuracy score . We want to make sure our ML classifiers outperform this baseline model.

In [18]:
dummy_classifier_accuracy = (36548 / (36548 + 4640))
dummy_classifier_accuracy

0.8873458288821987

In [19]:
from sklearn.dummy import DummyClassifier

dummy_classifier = DummyClassifier(strategy='constant', constant='no')

In [20]:
dummy_classifier.fit(X_train, y_train) # Does not actually do anything
dummy_classifier.score(X_train, y_train)

0.887119225664433

In [21]:
dummy_classifier.score(X_test, y_test)

0.8880256385354958

### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

In [22]:
import time
from sklearn.linear_model import LogisticRegression

# Automatically applies regularization
logistic_classifier = LogisticRegression()

In [23]:
X_train.head()

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,...,month_oct,month_sep,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success
549,0.957281,-0.406073,-0.565922,0.195414,-0.349494,0.648092,0.722722,0.886447,0.712460,0.331680,...,False,False,False,False,False,True,False,False,True,False
1974,-1.345734,-0.174659,0.517118,0.195414,-0.349494,0.648092,0.722722,0.886447,0.711307,0.331680,...,False,False,True,False,False,False,False,False,True,False
20992,-0.578062,-0.668342,0.156105,0.195414,-0.349494,0.839061,-0.227465,0.951267,0.774152,0.845170,...,False,False,False,False,True,False,False,False,True,False
1730,-0.961898,0.245742,-0.565922,0.195414,-0.349494,0.648092,0.722722,0.886447,0.711307,0.331680,...,False,False,True,False,False,False,False,False,True,False
26913,-1.057857,0.319023,-0.204909,0.195414,-0.349494,-0.115781,-0.649003,-0.323542,0.262167,0.398115,...,False,False,False,False,True,False,False,False,True,False


In [24]:
start_time = time.time()
logistic_classifier.fit(X_train, y_train)
lr_fit_time = time.time() - start_time

In [25]:
logistic_classifier.score(X_train, y_train)

0.91162474507138

In [26]:
logistic_classifier.score(X_test, y_test)

0.9124987860541905

Curiously, my logistic regression model actually performs (very slightly) better on the test set than on the training set. I guess I should at least take this as evidence that the model is, at least, not highly overfit.

### Problem 9: Score the Model

What is the accuracy of your model?

Included above.

### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

In [27]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

knn_classifier = KNeighborsClassifier()
decision_tree_classifier = DecisionTreeClassifier()
svm_classifier = SVC()

In [28]:
start_time = time.time()
knn_classifier.fit(X_train, y_train)
knn_fit_time = time.time() - start_time

In [29]:
start_time = time.time()
decision_tree_classifier.fit(X_train, y_train)
tree_fit_time = time.time() - start_time

In [30]:
start_time = time.time()
svm_classifier.fit(X_train, y_train)
svc_fit_time = time.time() - start_time

In [33]:
pd.DataFrame(
    {
        'Model': ['Logistic Regression', 'K-Nearest Neighbors', 'Decision Tree', 'SVC'],
        'Train Time': [lr_fit_time, knn_fit_time, tree_fit_time, svc_fit_time],
        'Train Accuracy': [logistic_classifier.score(X_train, y_train), knn_classifier.score(X_train, y_train),
                           decision_tree_classifier.score(X_train, y_train), svm_classifier.score(X_train, y_train)],
        'Test Accuracy': [logistic_classifier.score(X_test, y_test), knn_classifier.score(X_test, y_test),
                           decision_tree_classifier.score(X_test, y_test), svm_classifier.score(X_test, y_test)]
    }
)

,Model,Train Time,Train Accuracy,Test Accuracy
0,Logistic Regression,0.099401,0.911625,0.912499
1,K-Nearest Neighbors,0.022001,0.927390,0.903370
2,Decision Tree,0.138503,1.000000,0.886569
3,SVC,5.483080,0.922502,0.913567


### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.

- More feature engineering and exploration.  For example, should we keep the gender feature?  Why or why not?
- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric

##### Questions